# Initiliation

In [1]:
!pip install transformers datasets --quiet

In [2]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK


In [5]:
dfPJ = pd.read_csv('MBTI_PJ_URL_MASK.csv', sep=",", error_bad_lines=False)
dfPJ

,text,label
0,i like that you are kind as [MASK] i find that...,0
1,oh my you are right who really talks like tha...,1
2,yep yep yep especially the last one yep agree ...,0
3,things that are generalizable to the entire po...,1
4,work student hobbies studying gaming reading d...,1
...,...,...
6935,well mostly i dont like avocado but the primar...,1
6936,during an argument rather than trying to valid...,1
6937,cigarettes are like hamsters perfectly harmles...,1
6938,bookshelf porn as a nonamerican please excuse ...,1


# Model Training

In [6]:
modeltype = "princeton-nlp/sup-simcse-bert-base-uncased"

In [7]:
train, test = train_test_split(dfPJ, test_size=0.2, random_state=0, stratify=dfPJ.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_BERT_PJ_MASK", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Optimization

In [8]:
! pip install optuna --quiet

In [9]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [10]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "result/bert-base-uncased-cls_before_pooler-sym_mlp-mlp_bert-bs64-gpu8-gs1-lr5e-5-m=stsb-norm0.05-l32-contra",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
 

In [11]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_BERT_MASK_PJ"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-08 20:52:02,471] A new study created in RDB with name: SIMCSE_BERT_MASK_PJ


A new study created in RDB with name: SIMCSE_BERT_MASK_PJ


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "result/bert-base-uncased-cls_before_pooler-sym_mlp-mlp_bert-bs64-gpu8-gs1-lr5e-5-m=stsb-norm0.05-l32-contra",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.732300,0.690793,0.602305,0.375899,0.301153,0.500000
1,0.703600,0.673406,0.602305,0.375899,0.301153,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Evaluation *****
  Num examples = 1388
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.60      1.00      0.75       836
           1       0.00      0.00      0.00       552

    accuracy                           0.60      1388
   macro avg       0.30      0.50      0.38      1388
weighted avg       0.36      0.60      0.45      1388

[[836   0]
 [552   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

Trial 49 pruned. 


[I 2021-12-09 12:13:11,796] A new study created in memory with name: no-name-d3887b49-6163-469a-bc20-69f321e8efe4


A new study created in memory with name: no-name-d3887b49-6163-469a-bc20-69f321e8efe4


In [12]:
storage_name

'sqlite:///SIMCSE_BERT_MASK_PJ.db'

In [13]:
study_name

'SIMCSE_BERT_MASK_PJ'

In [14]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-09 12:13:11,913] Using an existing study with name 'SIMCSE_BERT_MASK_PJ' instead of creating a new one.


Using an existing study with name 'SIMCSE_BERT_MASK_PJ' instead of creating a new one.


In [15]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.375899,1.334051e-03,1.112470,4,8.452287e-08,COMPLETE
1,1,0.375899,1.706915e-02,2.957653,4,8.356806e-11,COMPLETE
2,2,0.375899,4.566845e-03,1.852005,2,2.341363e-05,COMPLETE
3,3,0.375899,2.098964e-01,6.310039,4,1.135784e-03,COMPLETE
4,4,0.375899,4.972884e-02,2.812853,4,1.731287e-05,COMPLETE
5,5,0.579870,1.511720e-05,3.080972,8,1.961919e-11,COMPLETE
6,6,0.284536,8.840512e-02,1.665303,16,9.903374e-09,PRUNED
7,7,0.375899,8.815628e-04,4.349901,4,6.833844e-10,PRUNED
8,8,0.375899,1.549912e-03,2.562984,2,3.888699e-07,COMPLETE
9,9,0.377593,7.913412e-08,1.988087,16,5.004756e-08,COMPLETE


In [16]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [17]:
best_run

BestRun(run_id='32', objective=0.5863922367954626, hyperparameters={'learning_rate': 4.83852752895182e-06, 'num_train_epochs': 6.760783415989672, 'per_device_train_batch_size': 8, 'weight_decay': 6.339600996727805e-11})

In [18]:
optuna.visualization.plot_intermediate_values(study)

In [19]:
optuna.visualization.plot_parallel_coordinate(study)

In [20]:
optuna.visualization.plot_optimization_history(study)

In [21]:
optuna.visualization.plot_contour(study)

In [22]:
optuna.visualization.plot_slice(study)

In [23]:
optuna.visualization.plot_edf(study)